In [2]:
import rasterio
import geopandas as gpd
import numpy as np
from skimage.graph.mcp import MCP_Geometric
from onsstove.raster import *

In [15]:
# This map represents the travel speed from this allocation process, expressed in units of minutes 
# required to travel one metre.
friction = r"C:\Users\camilorg\Box Sync\OnSTOVE\output\supply\LPG dealers\LPG dealers - friction - reprojected.tif"
# Startpoints
supply_points = r"C:\Users\camilorg\Box Sync\OnSTOVE\output\supply\LPG dealers\LPG dealers.gpkg"
gdf_supply_points = gpd.read_file(supply_points)
#distance raster
distance = r"output/supply/LPG dealers/LPG dealers - distance.tif"
#new friction based on start points
travel_time = r"output/supply/LPG dealers/LPG dealers - travel_time.tif"

In [4]:
# Read raster as array and set nodata to infinity to avoid taking nodata cells as path
with rasterio.open(friction) as src:
    fric = src.read(1)
    fric = fric*1000/60

In [8]:
fric[np.isnan(fric)] = float('inf')

In [9]:
fric

array([[inf, inf, inf, ..., inf, inf, inf],
       [inf, inf, inf, ..., inf, inf, inf],
       [inf, inf, inf, ..., inf, inf, inf],
       ...,
       [inf, inf, inf, ..., inf, inf, inf],
       [inf, inf, inf, ..., inf, inf, inf],
       [inf, inf, inf, ..., inf, inf, inf]], dtype=float32)

In [10]:
# Minimum cost path betweenn all nodes
mcp = MCP_Geometric(fric, fully_connected=True)

In [11]:
#Finding start points
row, col = friction_start_points(friction, supply_points) # travel_time does not need to be here, remove from function

In [12]:
#Adding statvalues
pointlist = []
i = 0
for test in row:
    positiions = [row[i],col[i]]
    pointlist.append(positiions)
    i = i+1
    
# pointlist = np.column_stack((row, col)) # suggestion

In [13]:
#minimum cost
cumulative_costs, traceback = mcp.find_costs(starts=pointlist)
cumulative_costs[np.where(cumulative_costs==float('inf'))] = np.nan

In [16]:
# the way this dataset is created can be improved
new_dataset = rasterio.open(travel_time, 'w', driver='GTiff',
                            height = src.shape[0], width = src.shape[1],
                            count=1, dtype="float64",
                            crs=src.crs, transform = src.transform)

new_dataset.write(cumulative_costs, 1)
new_dataset.close()